# LR 模型的实现

In [9]:
import numpy as np
import scipy 
import random

In [6]:
def sigmoid(x):
    

sigmoid(np.array([0, 4,5]))

array([0.5       , 0.98201379, 0.99330715])

In [21]:
a = np.random.random((2,3)) 
b = a
a,np.sum(a, axis=1)
#a+0
#a*b,a, np.dot(a,b.T)

(array([[0.33669245, 0.53688329, 0.57329556],
        [0.31298957, 0.89297023, 0.07174797]]),
 array([1.4468713 , 1.27770777]))

In [114]:
class LR():
    def __init__(self, lr = 0.01, L1=None, L2=None):
        self.W = None
        self.b = None
        self.epsilon = 0.0001
        self.L1_W = L1
        self.L2_W = L2
        self.X = None
        self.y = None
        self.y_hat = None
        self.lr = lr
        
    def sigmoid(self, X):
        return 1/(1 + np.exp(-X))
    
    
    def _get_gred(self):
        y_diff = self.y - self.y_hat
        W_gred = np.sum(y_diff * self.X, axis=0)
        b_gred = np.sum(y_diff)        
        return W_gred, b_gred
    
    def _init_param(self):
        self.W = np.random.random((self.X.shape[1],1)) 
        self.b = 0
    
    def fit(self, X, y):
        # 初始化参数
        self.X = X
        self.y = y
        self.y = self.y.reshape(y.shape[0],1)
        self._init_param()
        
        #print(self.X.shape)
        #print(self.W.shape)

        wx_b = np.dot(self.X , self.W) + self. b # n*m m*1
        #print(wx_b.shape,'wx')
        self.y_hat = self.sigmoid(wx_b)
        #print(self.y_hat.shape)
        
        W_gred, b_gred = self._get_gred()
        while(max(W_gred) > self.epsilon): # 迭代
            self.W = self.W  - self.lr * W_gred
            self.b = self.b - self.lr * b_gred
            #print('b2',self.b)
            wx_b = np.dot(self.X , self.W) + self. b # n*m m*1
            #print(wx_b.shape,'wx')

            self.y_hat = self.sigmoid(wx_b)
            
            W_gred, b_gred = self._get_gred()
        

In [115]:
lr = LR()
lr.fit(X_train, y_train)

## 标准的结果

In [40]:
from sklearn.datasets import load_iris
iris=load_iris()
X=iris.data
y=iris.target
y = y//2
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0) 
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [41]:
lr.predict_proba(X_test) # 查看第一个测试样本属于各个类别的概率

array([[4.41459348e-02, 9.55854065e-01],
       [8.80548636e-01, 1.19451364e-01],
       [9.99988369e-01, 1.16312153e-05],
       [1.07031153e-01, 8.92968847e-01],
       [9.99912357e-01, 8.76425006e-05],
       [1.78793744e-02, 9.82120626e-01],
       [9.99941029e-01, 5.89713401e-05],
       [8.24688831e-01, 1.75311169e-01],
       [7.89553704e-01, 2.10446296e-01],
       [9.06431875e-01, 9.35681248e-02],
       [1.19346744e-01, 8.80653256e-01],
       [8.38200035e-01, 1.61799965e-01],
       [6.88956224e-01, 3.11043776e-01],
       [7.50172366e-01, 2.49827634e-01],
       [6.25160419e-01, 3.74839581e-01],
       [9.99879244e-01, 1.20755658e-04],
       [6.50818979e-01, 3.49181021e-01],
       [5.68532507e-01, 4.31467493e-01],
       [9.99811097e-01, 1.88903169e-04],
       [9.99976951e-01, 2.30485427e-05],
       [1.06588471e-01, 8.93411529e-01],
       [5.18282353e-01, 4.81717647e-01],
       [9.99681094e-01, 3.18906359e-04],
       [9.99661004e-01, 3.38995707e-04],
       [3.815137

In [29]:
X.shape,y.shape

((150, 4), (150,))

In [27]:
iris_data[3]

array([4.6, 3.1, 1.5, 0.2])

In [36]:
2//2


1